In [1]:
pwd = %pwd
pwd[12:]

'PJATK/semestr4/ADD/Covid19_Prediction/notebooks'

In [2]:
import os
os.chdir("..")

In [3]:
pwd = %pwd
pwd[12:]

'PJATK/semestr4/ADD/Covid19_Prediction'

In [4]:
import pandas as pd

In [5]:
import json
with open("data/covid_data.json") as f:
    d = json.load(f)

In [6]:
df = pd.DataFrame(d["records"])

In [7]:
df.head()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2020,continentExp
0,23/10/2022,23,10,2022,3557.0,0.0,Austria,AT,AUT,8901064,Europe
1,22/10/2022,22,10,2022,5494.0,4.0,Austria,AT,AUT,8901064,Europe
2,21/10/2022,21,10,2022,7776.0,4.0,Austria,AT,AUT,8901064,Europe
3,20/10/2022,20,10,2022,8221.0,6.0,Austria,AT,AUT,8901064,Europe
4,19/10/2022,19,10,2022,10007.0,8.0,Austria,AT,AUT,8901064,Europe


In [8]:
"""todo create features -> 
is_święto, 
season(pora roku), 
death rate, 
R0 (https://en.wikipedia.org/wiki/Basic_reproduction_number)
"""

'todo create features -> \nis_święto, \nseason(pora roku), \ndeath rate, \nR0 (https://en.wikipedia.org/wiki/Basic_reproduction_number)\n'

In [9]:
df = df[df["countryterritoryCode"] == "POL"]
df['dateRep'] = pd.to_datetime(df['dateRep'], format="%d/%m/%Y")
df = df.sort_values("dateRep")


In [ ]:
from datetime import datetime, date
from holidays import country_holidays

list_of_holidays_in_poland = country_holidays('PL')

def check_if_holiday(day: int, month: int, year: int) -> int:
    if date(year, month, day) in list_of_holidays_in_poland: return int(True)
    return int(False)

def check_season(date_: datetime) -> str:
    spring_start = datetime(date_.year, 3, 21)
    summer_start = datetime(date_.year, 6, 22)
    autumn_start = datetime(date_.year, 9, 23)
    winter_start = datetime(date_.year, 12, 22)

    if date_ >= winter_start or date_ < spring_start: return "Winter"
    elif date_ >= spring_start and date_ < summer_start: return "Spring"
    elif date_ >= summer_start and date_ < autumn_start: return "Summer"
    elif date_ >= autumn_start and date_ < winter_start: return "Autumn"




df['is_holiday'] = df.apply(lambda row: check_if_holiday(int(row[1]), int(row[2]), int(row[3])), axis=1)
df['season'] = df.apply(lambda row: check_season(pd.to_datetime(row[0], format="%d/%m/%Y")), axis=1)
df['death_rate'] = df['deaths']/df['cases']  # tu mysle ze mozna sie pochylic nad braniem mianownika np. 5-7 dni bo po takim czasie czesto zgon nastepowal i bardziej dane do siebie sie odnosza

df

In [10]:
df["dateRep"]

23038   2020-03-04
23037   2020-03-05
23036   2020-03-06
23035   2020-03-07
23034   2020-03-08
           ...    
22077   2022-10-21
22076   2022-10-22
22075   2022-10-23
22074   2022-10-24
22073   2022-10-25
Name: dateRep, Length: 966, dtype: datetime64[ns]

In [11]:
cleaned_data = df.drop(["countryterritoryCode", "continentExp", "geoId", "countriesAndTerritories", "dateRep", "day", "month", "year", "popData2020", "deaths"], axis=1)

In [12]:
cleaned_data.head(5)

,cases
23038,1.0
23037,0.0
23036,4.0
23035,1.0
23034,5.0


In [13]:
cleaned_data['cases_next_day'] = df['cases'].shift(-1)

In [14]:
cleaned_data.head()

,cases,cases_next_day
23038,1.0,0.0
23037,0.0,4.0
23036,4.0,1.0
23035,1.0,5.0
23034,5.0,6.0


In [15]:
x = cleaned_data.drop(["cases_next_day"],  axis=1).values[:-1]
y = cleaned_data["cases_next_day"].values[:-1]

In [16]:
import pandas as pd
import numpy as np
import os
import typing
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2023-05-27 19:15:39.601236: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, shuffle=False)

In [18]:
# mean, std = x_train.mean(axis=0), x_train.std(axis=0)
# x_train = (x_train - mean) / std

In [19]:
# mean, std = x_val.mean(axis=0), x_val.std(axis=0)
# x_val = (x_val - mean) / std

In [20]:
from sklearn.ensemble import RandomForestRegressor

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [22]:
model = RandomForestRegressor(n_jobs=-1)
estimators = 100
model.set_params(n_estimators=estimators)


pipeline = Pipeline([
    ('scaler2' , StandardScaler()),
                        ('RandomForestRegressor: ', model)])
pipeline.fit(x_train , y_train)
prediction = pipeline.predict(x_val)

pipeline.fit(x_train, y_train)
pipeline.score(x_val, y_val)

0.8152480325755008

In [23]:
pp = pipeline.predict(x_val)

In [24]:
pipeline.predict([x_val[0]])

array([8981.74])

In [25]:
y_val[0]

11107.0

In [26]:
SEQUENCE_LENGTH=5

In [27]:
dataset_train = keras.preprocessing.timeseries_dataset_from_array(
    x_train,
    y_train,
    sequence_length=SEQUENCE_LENGTH
)

2023-05-27 19:15:47.604971: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [28]:
dataset_val = keras.preprocessing.timeseries_dataset_from_array(
    x_val,
    y_val,
    sequence_length=14,
#     sampling_rate=step,
#     batch_size=batch_size,
)


for batch in dataset_train.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Input value:", inputs.numpy()[0][0][0])
print("Input value:", inputs.numpy()[0][1][0])
# print("Input value:", inputs.numpy()[0][2][0])

print("Target shape:", targets.numpy().shape)
print("Target value:", targets.numpy()[0])

Input shape: (128, 5, 1)
Input value: 1.0
Input value: 0.0
Target shape: (128,)
Target value: 0.0


In [29]:
# inputs = keras.layers.Input(shape=(x_train.shape[1], 1))
# lstm_out = keras.layers.LSTM(32)(inputs)
# outputs = keras.layers.Dense(1)(lstm_out)

# model = keras.Model(inputs=inputs, outputs=outputs)
# model.compile(loss=tf.keras.losses.MeanSquaredError(),
#             optimizer=tf.keras.optimizers.Adam(),
#             metrics=[tf.keras.metrics.MeanAbsoluteError()])

# model.summary()

In [52]:
linear = tf.keras.Sequential([
    keras.layers.LSTM(32),
    tf.keras.layers.Dense(units=16, activation="relu"),
    tf.keras.layers.Dense(units=1)

    
])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=6,
                                                    mode='min')

linear.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

history = linear.fit(dataset_train, 
                     epochs=4000,
                     batch_size=300,
                     validation_data=dataset_val,
                     callbacks=[early_stopping])

Epoch 1/4000
6/6 [==============================] - 2s 85ms/step - loss: 106093920.0000 - mean_absolute_error: 6177.1753 - val_loss: 194750240.0000 - val_mean_absolute_error: 7128.8271
Epoch 2/4000
6/6 [==============================] - 0s 21ms/step - loss: 106090952.0000 - mean_absolute_error: 6176.9614 - val_loss: 194748368.0000 - val_mean_absolute_error: 7128.6904
Epoch 3/4000
6/6 [==============================] - 0s 20ms/step - loss: 106089344.0000 - mean_absolute_error: 6176.8262 - val_loss: 194746464.0000 - val_mean_absolute_error: 7128.5454
Epoch 4/4000
6/6 [==============================] - 0s 21ms/step - loss: 106087696.0000 - mean_absolute_error: 6176.6826 - val_loss: 194744496.0000 - val_mean_absolute_error: 7128.3765
Epoch 5/4000
6/6 [==============================] - 0s 18ms/step - loss: 106085744.0000 - mean_absolute_error: 6176.5107 - val_loss: 194741824.0000 - val_mean_absolute_error: 7128.1396
Epoch 6/4000
6/6 [==============================] - 0s 17ms/step - loss: 10

6/6 [==============================] - 0s 17ms/step - loss: 103848984.0000 - mean_absolute_error: 6006.2383 - val_loss: 192131312.0000 - val_mean_absolute_error: 6956.9912
Epoch 90/4000
6/6 [==============================] - 0s 16ms/step - loss: 103807216.0000 - mean_absolute_error: 6002.8345 - val_loss: 192082832.0000 - val_mean_absolute_error: 6954.5195
Epoch 91/4000
6/6 [==============================] - 0s 16ms/step - loss: 103765296.0000 - mean_absolute_error: 5999.8906 - val_loss: 192034192.0000 - val_mean_absolute_error: 6951.1694
Epoch 92/4000
6/6 [==============================] - 0s 18ms/step - loss: 103723176.0000 - mean_absolute_error: 5996.5625 - val_loss: 191985344.0000 - val_mean_absolute_error: 6948.8218
Epoch 93/4000
6/6 [==============================] - 0s 22ms/step - loss: 103680848.0000 - mean_absolute_error: 5994.2422 - val_loss: 191936176.0000 - val_mean_absolute_error: 6945.3228
Epoch 94/4000
6/6 [==============================] - 0s 18ms/step - loss: 103638312.

Epoch 133/4000
6/6 [==============================] - 0s 17ms/step - loss: 101697736.0000 - mean_absolute_error: 5869.9512 - val_loss: 189631584.0000 - val_mean_absolute_error: 6824.6279
Epoch 134/4000
6/6 [==============================] - 0s 16ms/step - loss: 101643176.0000 - mean_absolute_error: 5867.6816 - val_loss: 189569040.0000 - val_mean_absolute_error: 6822.2358
Epoch 135/4000
6/6 [==============================] - 0s 17ms/step - loss: 101588320.0000 - mean_absolute_error: 5863.6733 - val_loss: 189504272.0000 - val_mean_absolute_error: 6817.5264
Epoch 136/4000
6/6 [==============================] - 0s 16ms/step - loss: 101533464.0000 - mean_absolute_error: 5863.7227 - val_loss: 189440352.0000 - val_mean_absolute_error: 6815.1519
Epoch 137/4000
6/6 [==============================] - 0s 16ms/step - loss: 101477576.0000 - mean_absolute_error: 5856.9058 - val_loss: 189377296.0000 - val_mean_absolute_error: 6812.3867
Epoch 138/4000
6/6 [==============================] - 0s 17ms/ste

Epoch 177/4000
6/6 [==============================] - 0s 17ms/step - loss: 99066824.0000 - mean_absolute_error: 5727.4517 - val_loss: 186581424.0000 - val_mean_absolute_error: 6684.9751
Epoch 178/4000
6/6 [==============================] - 0s 17ms/step - loss: 99001832.0000 - mean_absolute_error: 5723.2622 - val_loss: 186505424.0000 - val_mean_absolute_error: 6681.8315
Epoch 179/4000
6/6 [==============================] - 0s 17ms/step - loss: 98936680.0000 - mean_absolute_error: 5720.7666 - val_loss: 186430640.0000 - val_mean_absolute_error: 6679.7437
Epoch 180/4000
6/6 [==============================] - 0s 16ms/step - loss: 98871248.0000 - mean_absolute_error: 5716.5015 - val_loss: 186354432.0000 - val_mean_absolute_error: 6675.9355
Epoch 181/4000
6/6 [==============================] - 0s 16ms/step - loss: 98805648.0000 - mean_absolute_error: 5713.1450 - val_loss: 186278784.0000 - val_mean_absolute_error: 6672.8936
Epoch 182/4000
6/6 [==============================] - 0s 17ms/step - l

In [50]:
lpred = linear.predict(x_val)

10/10 [==============================] - 0s 1ms/step


In [31]:
# https://www.kaggle.com/code/arpandas65/covid-19-projection-using-lstm
# https://deepbas.io/2022/03/23/how-to-predict-covid-case-counts-using-machine-learning-models/

In [32]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN

In [33]:
model = Sequential()

model.add(SimpleRNN(units=32, input_shape=(1, 1), activation="relu")) 
model.add(Dense(16, activation="relu")) 
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam') 
# model.compile(loss='mae', optimizer='adam') 
 
model.summary()
hist = model.fit(x_train, y_train,
                     epochs=100,
                )

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 32)                1088      
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1,633
Trainable params: 1,633
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
22/22 [==============================] - 1s 2ms/step - loss: 23147718.0000
Epoch 2/100
22/22 [==============================] - 0s 2ms/step - loss: 9298606.0000
Epoch 3/100
22/22 [==============================] - 0s 2ms/step - loss: 7105471.5000
Epoch 4/100
22/22 [==============================] - 0s 2

In [34]:
predicted = model.predict(x_val)

10/10 [==============================] - 0s 984us/step


In [35]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_val, predicted)

10137979.383510962

In [36]:
mean_squared_error(y_val, pp)

25702944.727488168

In [51]:
mean_squared_error(y_val, lpred.squeeze())

180747983.77253142

In [37]:
predicted[:10][9]

array([10035.951], dtype=float32)

In [38]:
y_val[:10][9]

19648.0